In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [4]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [5]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

### 1) Who are the top and bottom 10  users by 

In [34]:
### Parsing Data
data_directory = os.path.join(rpath, "data")
slack_data_loader = SlackDataLoader(data_directory)
# List all directories in the 'data' directory
directories = [d for d in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, d))]


dfs_by_directory = []

# Iterate through each directory
for directory in directories:
    directory_path = os.path.join(data_directory, directory)
    df_directory = slack_data_loader.slack_parser(directory_path)
    
    # Add a 'directory' column to identify the source directory
    df_directory['directory'] = directory
    
    # Append the DataFrame to the list
    dfs_by_directory.append(df_directory)

# Concatenate all DataFrames into a single DataFrame
slack_parser_df = pd.concat(dfs_by_directory, ignore_index=True)
slack_parser_df.head(5)

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel,directory
0,message,"Hi guys,\nI thought having a group chat would ...",Anita Rodriguez,1661766066.863109,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
1,message,good!!,Michael Gonzalez,1661766159.852649,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
2,message,Thanks <@U03U9FWPNCE>,Robert Carter,1661766226.215889,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
3,message,Nice <@U03U9FWPNCE>,Phillip Atkins,1661766241.597379,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
4,message,I have been missing deadlines and I am trying ...,Anita Rodriguez,1661766295.939119,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group


#### a) by reply count

In [7]:

# Users grouped by 'sender_name
grouped_by_user = slack_parser_df.groupby('sender_name')['reply_users_count'].sum().reset_index(name='total_reply_count')

In [8]:
# TOP 10 users by reply count
sorted_by_reply_count = grouped_by_user.sort_values(by='total_reply_count', ascending=False)
sorted_by_reply_count.head(10)

,sender_name,total_reply_count
15,Cristian Wilson,311
57,Vanessa Norman,276
42,Michelle Lewis,193
8,Brenda Hernandez,179
9,Brian Odom,171
56,Travis Butler,162
4,April Lucas,153
1,Amy Leon,151
30,Joshua Rhodes,143
49,Phillip Atkins,138


In [9]:
# BOTTOM 10 users by reply count
sorted_by_reply_count = grouped_by_user.sort_values(by='total_reply_count', ascending=True)
sorted_by_reply_count.head(10)

,sender_name,total_reply_count
22,Erica Munoz,0
46,Patricia Thomas,0
6,Blake Scott,0
54,Slackbot,0
47,Patrick Hernandez,0
25,Holly Whitaker,0
17,David Scott,0
14,Christopher Case,0
35,Leah Newton,2
50,Randy Miller,3


#### b) by mentions

In [26]:
filtered_df = slack_parser_df[slack_parser_df['msg_content'].str.contains('@U', regex=True)]
filtered_df.head(5)

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel,directory
0,message,"Hi guys,\nI thought having a group chat would ...",Anita Rodriguez,1661766066.863109,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
2,message,Thanks <@U03U9FWPNCE>,Robert Carter,1661766226.215889,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
3,message,Nice <@U03U9FWPNCE>,Phillip Atkins,1661766241.597379,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
6,message,In the mean time lets all read the document th...,Phillip Atkins,1661766471.373039,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
14,message,Just created the repo <@U03U9FWPNCE>,Robert Carter,1661767920.902709,text,0,0,0,0,0,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group


In [58]:
count_by_sender = filtered_df.groupby('sender_name').size().reset_index(name='mentions_count')
sorted_mention_counts = count_by_sender.sort_values(by='mentions_count', ascending=False)

# Top 10 users by mentions
top_10_mentions = sorted_mention_counts.head(10)
top_10_mentions

,sender_name,mentions_count
49,Vanessa Norman,260
6,Brady Rhodes,162
48,Travis Butler,147
1,Amy Leon,137
13,Cristian Wilson,134
4,April Lucas,132
43,Phillip Atkins,128
7,Brenda Hernandez,125
26,Judith Bolton,112
8,Brian Odom,103


In [13]:
sorted_mention_counts = count_by_sender.sort_values(by='mentions_count', ascending=True)

# Bottom users 10 mentions
bottom_10_mentions = sorted_mention_counts.head(10)
bottom_10_mentions

,sender_name,mentions_count
30,Leah Newton,1
42,Patrick Hernandez,1
45,Randy Miller,1
41,Patricia Thomas,1
49,Steven Garcia,2
36,Michele Sparks,3
18,Eric Myers,4
23,John Romero,4
16,Dustin Wilson,5
11,Carol Kaiser,5


#### c) by messages

In [59]:
count_by_sender = slack_parser_df.groupby('sender_name').size().reset_index(name='messages_count')
sorted_messages_counts = count_by_sender.sort_values(by='messages_count', ascending=False)

# Top 10 users messages
top_10_messages = sorted_messages_counts.head(10)
top_10_messages

,sender_name,messages_count
57,Vanessa Norman,1288
1,Amy Leon,1150
56,Travis Butler,1075
4,April Lucas,991
8,Brenda Hernandez,843
15,Cristian Wilson,831
49,Phillip Atkins,754
7,Brady Rhodes,641
31,Judith Bolton,574
30,Joshua Rhodes,519


In [57]:
count_by_sender = slack_parser_df.groupby('sender_name').size().reset_index(name='messages_count')
sorted_messages_counts = count_by_sender.sort_values(by='messages_count', ascending=True)

# Bottom 10 users by messages
bottom_10_messages = sorted_messages_counts.head(10)
bottom_10_messages

,sender_name,messages_count
6,Blake Scott,1
22,Erica Munoz,1
17,David Scott,1
25,Holly Whitaker,1
14,Christopher Case,1
54,Slackbot,3
47,Patrick Hernandez,5
50,Randy Miller,7
46,Patricia Thomas,8
35,Leah Newton,10


#### d) by reactions

In [14]:
data_directory = os.path.join(rpath, "data")
slack_data_loader = SlackDataLoader(data_directory)
# List all directories in the 'data' directory
directories = [d for d in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, d))]


df_by_directory = []

# Iterate through each directory
for directory in directories:
    directory_path = os.path.join(data_directory, directory)
    df_directory = slack_data_loader.parse_slack_reaction(directory_path)
    
    # Add a 'directory' column to identify the source directory
    df_directory['directory'] = directory
    
    # Append the DataFrame to the list
    df_by_directory.append(df_directory)

# Concatenate all DataFrames into a single DataFrame
reaction_df = pd.concat(df_by_directory, ignore_index=True)

In [15]:
reaction_df.head(5)

,reaction_name,reaction_count,reaction_users_count,message,user_id,channel,directory
0,+1,1,U03U9FWPNCE,So I'm going to create a GitHub organization. ...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
1,+1,1,U03UG4Q7V42,So I would fork the repo to my GitHub then clo...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
2,raised_hands,2,"U03UG4Q7V42,U03U1HAG9TR",Yaa!,U03UD4FEDHB,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
3,+1,2,"U03UG4Q7V42,U03U1HAG9TR","Sorry guys for my delay, i joind",U03UD4FEDHB,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group
4,+1,1,U03UG4Q7V42,Next meetup will be on Slack @ around 10 p.m.:...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group


In [16]:
users_df= pd.DataFrame(slack_data_loader.get_users())
users_df.rename(columns={'id':'user_id'}, inplace = True)
users_df.columns

Index(['user_id', 'team_id', 'name', 'deleted', 'color', 'real_name', 'tz',
       'tz_label', 'tz_offset', 'profile', 'is_admin', 'is_owner',
       'is_primary_owner', 'is_restricted', 'is_ultra_restricted', 'is_bot',
       'is_app_user', 'updated', 'is_email_confirmed',
       'who_can_share_contact_card', 'is_invited_user'],
      dtype='object')

In [37]:
merged_df = pd.merge(reaction_df, users_df, on='user_id', how='outer')
merged_df.head(5)

,reaction_name,reaction_count,reaction_users_count,message,user_id,channel,directory,team_id,name,deleted,...,is_owner,is_primary_owner,is_restricted,is_ultra_restricted,is_bot,is_app_user,updated,is_email_confirmed,who_can_share_contact_card,is_invited_user
0,+1,1,U03U9FWPNCE,So I'm going to create a GitHub organization. ...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
1,+1,1,U03UG4Q7V42,So I would fork the repo to my GitHub then clo...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
2,+1,1,U03UG4Q7V42,Next meetup will be on Slack @ around 10 p.m.:...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
3,+1,1,U03U9FWPNCE,<@U03U9FWPNCE> It's you're task:\n*CML helps t...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
4,white_check_mark,2,"U03UG4Q7V42,U03UD4FEDHB",<@U03UG4Q7V42> For each version of the data cr...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN


In [18]:

# Users grouped by 'sender_name
grouped_by_reaction = merged_df.groupby('real_name')['reaction_count'].sum().reset_index(name='total_reaction_count')

In [19]:
# TOP 10 users by reaction count
sorted_by_reaction_count = grouped_by_reaction.sort_values(by='total_reaction_count', ascending=False)
sorted_by_reaction_count.head(10)

,real_name,total_reaction_count
66,Vanessa Norman,1124
4,April Lucas,625
1,Amy Leon,573
36,Judith Bolton,536
18,Cristian Wilson,509
9,Brady Rhodes,448
65,Travis Butler,440
58,Phillip Atkins,428
11,Brian Odom,424
37,Katherine Foster,383


In [20]:
# BOTTOM 10 users by reaction count
sorted_by_reaction_count = grouped_by_reaction.sort_values(by='total_reaction_count', ascending=True)
sorted_by_reaction_count.head(10)

,real_name,total_reaction_count
26,Erica Munoz,0
56,Paul Brown,0
38,Kathleen Phillips,0
21,David Scott,0
19,Dana Parks,0
61,Ryan Cochran,0
17,Claudia Riddle,0
41,Kenneth Fitzgerald,0
30,Holly Whitaker,0
29,Glen Pennington,0


### 2) What are the top 10 messages by

#### a) Replies?

In [49]:

# Users grouped by 'sender_name
grouped_by_message = slack_parser_df.groupby('msg_content').size().reset_index(name='total_reply_count')

In [50]:
# TOP 10 messages by reply count
sorted_by_reply_count = grouped_by_message.sort_values(by='total_reply_count', ascending=False)
sorted_by_reply_count.head(10)

,msg_content,total_reply_count
5780,Hello,150
15582,yes,89
13076,hello,57
11574,Yes,57
927,:rolling_on_the_floor_laughing:,40
5991,Hi,35
265,*Independent challenge work Reminder!! <!here>...,33
5440,Good morning,33
10120,Thanks,27
730,:+1:,26


#### b) Reactions?

In [38]:
merged_df.head(5)

,reaction_name,reaction_count,reaction_users_count,message,user_id,channel,directory,team_id,name,deleted,...,is_owner,is_primary_owner,is_restricted,is_ultra_restricted,is_bot,is_app_user,updated,is_email_confirmed,who_can_share_contact_card,is_invited_user
0,+1,1,U03U9FWPNCE,So I'm going to create a GitHub organization. ...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
1,+1,1,U03UG4Q7V42,So I would fork the repo to my GitHub then clo...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
2,+1,1,U03UG4Q7V42,Next meetup will be on Slack @ around 10 p.m.:...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
3,+1,1,U03U9FWPNCE,<@U03U9FWPNCE> It's you're task:\n*CML helps t...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN
4,white_check_mark,2,"U03UG4Q7V42,U03UD4FEDHB",<@U03UG4Q7V42> For each version of the data cr...,U03U1HAG9TR,c:\Users\user\Desktop\Week 0\Slack_message_ana...,ab_test-group,T03U4J8HMUG,Robert,False,...,False,False,False,False,False,False,1685379067,True,EVERYONE,NaN


In [45]:

# Messages grouped by 'message'
grouped_by_message = merged_df.groupby('message').size().reset_index(name='total_reaction_count')

In [46]:
# TOP 10 messages by reaction count
sorted_by_reaction_count = grouped_by_message.sort_values(by='total_reaction_count', ascending=False)
sorted_by_reaction_count.head(10)

,message,total_reaction_count
0,,175
1211,Done,8
553,<@U03UJGP0C68> <@U03UUR571A5> <@U03V785NLSU> p...,7
322,<!channel>\nThere will be no careers exercise ...,7
99,*Independent challenge work Reminder!! <!here>...,5
1370,Hachalu Hundessa - Maalan Jira! **NEW**2015**,5
2133,Jurassic world dominion,5
317,:wink:,5
226,8,5
1474,"Hey guys, Am back, with new energy",5


#### c) Mentions?

In [53]:
count_by_msg = filtered_df.groupby('msg_content').size().reset_index(name='mentions_count')
sorted_mention_counts = count_by_msg.sort_values(by='mentions_count', ascending=False)

# Top 10 messages by mentions
top_10_mentions = sorted_mention_counts.head(10)
top_10_mentions

,msg_content,mentions_count
225,<@U03TEPYRM2P>,6
367,<@U03U93GNNVB>,6
1288,<@U03V1AM5TFA>,6
992,<@U03UKL27B0R>,6
1050,<@U03UP7V9Q57>,4
821,<@U03UJGP0C68>,4
212,<@U03T89ACUUW>,4
403,<@U03U9EJR362>,3
2013,Thank you <@U03UVHCV6KB>,3
967,<@U03UJN29Y4C>,3


In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [11]:
# what kind of messages are replied faster than others?

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?